In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pymc as pm

sns.set()

In [ ]:
scb = pd.read_csv('scb-dead_per_day.csv',sep=';',usecols=[1,2,3,4,5,6,7])
scb.index = pd.date_range('2020-01-01','2020-12-31')
scb.drop(pd.Timestamp('2020-02-29'),inplace=True,axis=0)
last_day = scb.loc[scb['2020'] == 0]
last_day = last_day.iloc[0]
last_day = last_day.name.date()
last_data_day = last_day - pd.DateOffset(days=1)

last_day = (last_day - pd.DateOffset(days=7)).date() # skip last 6 days; last day is one day after before call


In [ ]:
daily_min = scb.loc[:,'2015' : '2019'].min(axis=1)
daily_max = scb.loc[:,'2015' : '2019'].max(axis=1)
daily_mean = scb.loc[:,'2015' : '2019'].mean(axis=1)
cum_diff = scb.loc[:last_day,'2020'].sum() - scb.loc[:last_day,'2015-2019'].sum()
cum_diff

In [ ]:
excess = scb.loc[:last_day,'2020'] - scb.loc[:last_day,'2015-2019']
excess

In [ ]:
plt.figure(figsize=(18,12))
ax = plt.gca()
ax2 = plt.twinx()

ax.plot(scb['2015-2019'],'o',color='lime',label='baseline 2015-2019')
ax.fill_between(scb.index,y1=daily_min,y2=daily_max,color='olive',alpha=0.2,label='baseline range')
ax.plot(scb.loc[:last_day,'2020'],'o',color='crimson',label='2020')
ax.set_title('SWEDEN daily deaths : 2020-01-01 - {} total diff in deaths Y2D: {}\nData Source: SCB.se'.format(last_day,cum_diff))
ax.set_ylabel('number of daily deaths')
ax.legend(loc='upper left')
ax2.plot(excess.cumsum(),ls='dashed',color='orange',label='Excess Deaths Y2D')
ax2.set_ylabel('Total Excess deaths')
ax2.legend(loc='upper right')

plt.savefig('SCB_compare_with_interval.jpg',format='jpg')

In [ ]:
scb_weekly = scb.resample('W-TUE').sum()
scb_weekly.loc[:last_data_day]

In [ ]:
weekly_min = scb_weekly.loc[:,'2015' : '2019'].min(axis=1)
weekly_max = scb_weekly.loc[:,'2015' : '2019'].max(axis=1)
weekly_mean = scb_weekly.loc[:,'2015' : '2019'].mean(axis=1)
weekly_cum_diff = scb_weekly.loc[:last_data_day,'2020'].sum() - scb_weekly.loc[:last_data_day,'2015-2019'].sum()
weekly_cum_diff

In [ ]:
weekly_excess = scb_weekly.loc[:last_data_day,'2020'] - scb_weekly.loc[:last_data_day.date(),'2015-2019']
weekly_excess

In [ ]:
last_week = scb_weekly.loc[:last_data_day].index[-1].date()
last_week

In [ ]:
plt.figure(figsize=(18,12))
ax = plt.gca()
ax2 = plt.twinx()

ax.plot(scb_weekly['2015-2019'][:-1],'o--',color='lime',label='baseline 2015-2019')
ax.fill_between(scb_weekly.index[:-1],y1=weekly_min[:-1],y2=weekly_max[:-1],color='olive',alpha=0.2,label='baseline range')
ax.plot(scb_weekly.loc[:last_week,'2020'],'o--',color='crimson',label='2020')
ax.set_title('SWEDEN weekly deaths : 2020-01-01 - {} total diff in deaths Y2D: {}\nData Source: SCB.se'.format(last_week,
                                                                                                               weekly_cum_diff))
ax.set_ylabel('number of daily deaths')
ax.legend(loc='upper left')
ax2.plot(weekly_excess.cumsum(),ls='dashed',color='orange',label='Excess Deaths Y2D')
ax2.set_ylabel('Total Excess deaths')
ax2.legend(loc='upper right')

plt.savefig('SCB_compare_weekly_with_interval.jpg',format='jpg')